In [4]:
import boto3
import os
import tempfile
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

def load_roberta_from_s3(bucket_path):
    # Séparer bucket et prefix
    if "/" not in bucket_path:
        raise ValueError("Le chemin doit être de la forme 'bucket/prefix'")
    bucket_name, prefix = bucket_path.split("/", 1)
    prefix = prefix.rstrip("/")  # enlever / final si présent

    s3 = boto3.client("s3")
    tmp_dir = tempfile.mkdtemp()
    print(f"Dossier temporaire: {tmp_dir}")

    # Lister les fichiers dans le prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    if "Contents" not in response:
        raise FileNotFoundError(f"Aucun fichier trouvé dans s3://{bucket_name}/{prefix}")

    # Télécharger tous les fichiers
    for obj in response["Contents"]:
        key = obj["Key"]
        if key.endswith("/"):
            continue
        local_path = os.path.join(tmp_dir, os.path.relpath(key, prefix))
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        print(f"Téléchargement: s3://{bucket_name}/{key} -> {local_path}")
        s3.download_file(bucket_name, key, local_path)

    # Charger modèle et tokenizer
    tokenizer = RobertaTokenizer.from_pretrained(tmp_dir)
    model = RobertaForSequenceClassification.from_pretrained(tmp_dir)

    return tokenizer, model

# ===== Utilisation =====
bucket_path = "sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/"

tokenizer, model = load_roberta_from_s3(bucket_path)

Dossier temporaire: /tmp/tmpdxu3_htb
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/config.json -> /tmp/tmpdxu3_htb/config.json
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/merges.txt -> /tmp/tmpdxu3_htb/merges.txt
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/model.safetensors -> /tmp/tmpdxu3_htb/model.safetensors
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/optimizer.pt -> /tmp/tmpdxu3_htb/optimizer.pt
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/rng_state.pth -> /tmp/tmpdxu3_htb/rng_state.pth
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/scaler.pt -> /tmp/tmpdxu3_htb/scaler.pt
Téléchargement: s3://sagemaker-studio-oxs6vznjds/writing_task_models/accuracy/checkpoint-1800/scheduler.pt -> /tmp/tmpdxu3_

In [5]:
def format_text_inference(ef_level, activity_instructions, student_submission):
    return (
        f"Prompt Level: {ef_level} [SEP] Prompt: {activity_instructions} [SEP] Response: {student_submission}"
    )

In [6]:
def inference(input_json):
    """
    input_json attendu :
    {
      "answer": "...",
      "prompt": "...",
      "level": "..."  # chaîne ou int
    }
    """
    # Extraire les champs
    ef_level = int(input_json["ef_level"])
    activity_instructions = input_json["activity_instructions"]
    student_submission = input_json["student_submission"]
    
    # Formater le texte
    formatted_text = format_text_inference(ef_level, activity_instructions, student_submission)
    
    # Tokenizer
    inputs = tokenizer(formatted_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1).squeeze()
    predicted_class = torch.argmax(probs).item()

    mapped_score = map_score_linear(predicted_class)

    
    # Construire la sortie JSON
    output = {
        "cefr_scoring": predicted_class,
        "cefr_scoring_100": mapped_score,
        "scorer": {
            "version": "roberta_large_onnx_scorer",
            "release": "0.3"
        }
    }
    return output


def map_score_linear(score):
    evp_to_score = {
        0: 17,
        1: 33,
        2: 50,
        3: 67,
        4: 83,
        5: 100,
    }
    return evp_to_score.get(score)


In [7]:
example_input = {
    "ef_level": 10,
    "activity_instructions": "Read the email from your manager. Then respond with an email that has several ideas to help her solve the budget problem. Type in the input box. Write between 80 and 100 words. Use your own words where possible. ",
    "student_submission": "Response: Hi Carla,\n\nThe financial report was shocking. We have a budget crisis and I have a list of options how to deal with this crisis on a long-team basis. \n\n-First I would recommend that we would cut down everyone’s working hours. The company would save about $10000 per worker each year. \n-Secondly we should think about offering older workers a large retirement bonus if they accept our resignation package. If we lay off senior workers we could save about $300 000 every year.\n-Thirdly I would also recommend updating our offices to present-day. We have many offices which are too huge and expensive and old-fashioned. If we move office space to another location we could save money in rent. By changing location we could possibly save about $10000"
}

result = inference(example_input)
print(result)

{'cefr_scoring': 4, 'cefr_scoring_100': 83, 'scorer': {'version': 'roberta_large_onnx_scorer', 'release': '0.3'}}


## onnx inference

In [5]:
import boto3
import onnxruntime as ort

# Define S3 bucket and model key
bucket_name = 'sagemaker-studio-oxs6vznjds'
model_key = 'writing_task_models/accuracy/model_1800_roberta_large.onnx'
local_model_path = '/tmp/roberta-large-ft-acc-writing-task-1800.onnx'  # or wherever you want to save temporarily

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Download the ONNX model from S3 to local path
s3.download_file(bucket_name, model_key, local_model_path)

# Load the ONNX model using onnxruntime
session = ort.InferenceSession(local_model_path)

print("ONNX model loaded successfully.")

ONNX model loaded successfully.


In [14]:
max_length = 256  # Ajuste selon la taille maximale de ton modèle
import torch.nn.functional as F  # pour softmax
import numpy as np

def inference(input_json, onnx_model):
    ef_level = int(input_json["ef_level"])
    activity_instructions = input_json["activity_instructions"]
    student_submission = input_json["student_submission"]
    
    formatted_text = format_text_inference(ef_level, activity_instructions, student_submission)
    
    inputs = tokenizer(
        formatted_text, 
        padding=True, 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
    )
    
    input_ids = inputs["input_ids"].cpu().numpy()
    attention_mask = inputs["attention_mask"].cpu().numpy()
    onnx_inputs = {"input_ids": input_ids, "attention_mask": attention_mask}
    
    onnx_outputs = onnx_model.run(None, onnx_inputs)
    logits = onnx_outputs[0]
    
    predicted_class = int(np.argmax(logits, axis=1)[0])
    
    probs = F.softmax(torch.tensor(logits), dim=1).numpy().squeeze()
    predicted_prob = float(probs[predicted_class])
    
    mapped_score = map_score_linear(predicted_class)
    
    output = {
        "cefr_scoring": predicted_class,
        "cefr_scoring_100": mapped_score,
        "predicted_probability": round(predicted_prob, 2),
        "scorer": {
            "version": "roberta_large_onnx_scorer",
            "release": "0.1"
        }
    }
    return output

def map_score_linear(score):
    evp_to_score = {
        0: 17,
        1: 33,
        2: 50,
        3: 67,
        4: 83,
        5: 100,
    }
    return evp_to_score.get(score)


In [15]:
example_input = {
    "ef_level": 10,
    "activity_instructions": "Read the email from your manager. Then respond with an email that has several ideas to help her solve the budget problem. Type in the input box. Write between 80 and 100 words. Use your own words where possible. ",
    "student_submission": "Response: Hi Carla,\n\nThe financial report was shocking. We have a budget crisis and I have a list of options how to deal with this crisis on a long-team basis. \n\n-First I would recommend that we would cut down everyone’s working hours. The company would save about $10000 per worker each year. \n-Secondly we should think about offering older workers a large retirement bonus if they accept our resignation package. If we lay off senior workers we could save about $300 000 every year.\n-Thirdly I would also recommend updating our offices to present-day. We have many offices which are too huge and expensive and old-fashioned. If we move office space to another location we could save money in rent. By changing location we could possibly save about $10000"
}

result = inference(example_input, onnx_model=session)
print(result)

{'cefr_scoring': 4, 'cefr_scoring_100': 83, 'predicted_probability': 0.88, 'scorer': {'version': 'roberta_large_onnx_scorer', 'release': '0.1'}}
